In [1]:
import pandas as pd
import os
import pickle
import tweepy_helpers as th

In [2]:
# set up working directory
os.path.abspath(os.getcwd()) # initial working directory (should be equal to source file directory if using Jupyter Notebook)
os.chdir('../../data/web_scraping') # change to directory where all data files are stored
# check working directory
os.path.abspath(os.getcwd())

'C:\\Users\\Simon\\OneDrive\\Uni\\LMU\\SS 2020\\Statistisches Consulting\\Bundestag-MP-Analyse\\data\\web_scraping'

In [3]:
# import Bundestag data
with open('abg_df.pickle', 'rb') as handle:
    bt_data = pickle.load(handle)
    
# select name and username for each member and store in table called twitter_account
names = bt_data['Name']
twitter_usernames = bt_data['Twitter']
names.rename("name", inplace = True)
twitter_usernames.rename("username", inplace = True)
twitter_account = pd.concat([names, twitter_usernames], axis = 1)

# drop usernames that are nan or empty (i.e. parliamentarians with no account)
usr_nan = twitter_account.username.isna()
usr_empty = twitter_account.username == ''
twitter_account = twitter_account[~(usr_nan | usr_empty)]
twitter_account.reset_index(drop = True, inplace = True)

In [ ]:
# download most recent tweets using tweepy (at most 3200 tweets per user)
tweepy_df = pd.DataFrame()
for username in twitter_account['username']:
    tweepy_df = pd.concat([tweepy_df, th.download_tweets_tweepy(username)])
# add column 'name'
tweepy_df = twitter_account.merge(tweepy_df, on = 'username')

# save data
with open('tweepy_df.pickle', 'wb') as handle:
    pickle.dump(tweepy_df, handle, protocol=pickle.HIGHEST_PROTOCOL)